TTGO or Heltec receive any packet and display it on the OLED

Depends on MicroPython for ESP32, 
and https://github.com/goatchurchprime/LightLora_MicroPython
and ssd1306.py OLED library

Useful set of documentes: https://github.com/Wei1234c/MQTT_bridged_LoRa_networks_with_MicroPython_on_ESP32/tree/master/references

In [2]:
# Run this to upload the files
%serialconnect
%sendtofile --quiet --source ssd1306.py
%sendtofile --mkdir --quiet --source /home/julian/extrepositories/LightLora_MicroPython/LightLora/ lib/LightLora/



***Connection broken [Input/output error]
You may need to reconnectserial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.
Sent 168 lines (5477 bytes) to ssd1306.py.
Sent 106 lines (2864 bytes) to lib/LightLora/lorautil.py.
Sent 397 lines (13311 bytes) to lib/LightLora/sx127x.py.
Sent 64 lines (2087 bytes) to lib/LightLora/spicontrol.py.
Sent 0 lines (0 bytes) to lib/LightLora/__init__.py.


In [55]:
# start here for interaction
%serialconnect



***Connection broken [Input/output error]
You may need to reconnectserial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB4 --baud=115200 
Ready.


In [18]:
%sendtofile main.py

from machine import Pin, I2C
import ssd1306

oledrst = Pin(16, Pin.OUT)
oledrst.value(1)
i2c = I2C(scl=Pin(15), sda=Pin(4), freq=450000)
o = ssd1306.SSD1306_I2C(128, 64, i2c, addr=0x3c)

o.fill(0)
o.text('Connecting to', 0, 0)
o.text('Lora', 0, 8)
o.show() 

Sent 12 lines (257 bytes) to main.py.


In [57]:

from LightLora import spicontrol, sx127x
spic = spicontrol.SpiControl()
spic.initLoraPins()

params = {  'frequency': 868100000, 
            'tx_power_level': 14, 
            'signal_bandwidth': 125E3, 
            'spreading_factor': 7, 
            'coding_rate': 5, 
            'preamble_length': 8,
            'implicitHeader': False, 
            'sync_word': 0x34,    # from _loraModem
            'enable_CRC': True }

lora = sx127x.SX127x(spiControl=spic, parameters=params)


Traceback (most recent call last):
  File "<stdin>", line 18, in <module>
NameError: name 'lorautil' is not defined


In [60]:
lora.init()


In [71]:
pkt = { }
def _doReceive(sx12, pay):
    self.packet = None
    if pay and len(pay) > 4:
        pkt["srcAddress"] = pay[0]
        pkt["dstAddress"] = pay[1]
        pkt["srcLineCount"] = pay[2]
        pkt["payLength"] = pay[3]
        pkt["rssi"] = sx12.packetRssi()
        pkt["snr"] = sx12.packetSnr()
        try:
            pkt["msgTxt"] = pay[4:].decode('utf-8', 'ignore')
        except Exception as ex:
            print("doReceiver error: ")
            print(ex)

doneTransmit = False
def _doTransmit():
    global doneTransmit
    doneTransmit = True
    lora.receive() # wait for a packet (?)
            
lora.onReceive(_doReceive)
lora.onTransmit(_doTransmit)
lora.receive()


[leftinbuffer] ['Traceback (most recent call last):']
[leftinbuffer] ['  File "/lib/LightLora/sx127x.py", line 342, in _handleOnTransmit']
[leftinbuffer] ['TypeError: function takes 1 positional arguments but 0 were given']


In [64]:
print(pkt)


{}


In [67]:
linecounter = 0

def sendPacket(dstAddress, localAddress, outGoing):
    global linecounter, doneTransmit
    linecounter = linecounter + 1
    doneTransmit = False
    lora.beginPacket()
    lora.write(bytearray([dstAddress, localAddress, linecounter, len(outGoing)]))
    lora.write(outGoing)
    lora.endPacket()

import time
def waitforsendpacket():
    slt = 0
    while (not doneTransmit) and (slt < 50):
        time.sleep(.1)
        slt = slt + 1
    if slt == 50:
        print("Transmit timeout")

               


In [77]:
btxt = b"\x90\x91\x92"
sendPacket(0xfe, 0x41, btxt) 


In [19]:
%sendtofile --append main.py

from lib.LightLora import lorautil, spicontrol
lru = lorautil.LoraUtil()

Sent 2 lines (72 bytes) to main.py.


In [54]:
lru.lora.setSyncWord(0x36)

In [53]:
n = 1999
txt = "Hello %d there"%n
btxt = txt.encode()
btxt = b"\x90\x91\x92"
lru.sendPacket(0xfe, 0x41, btxt) 


In [32]:
%sendtofile --append main.py

import time

n = 0
while True:
    time.sleep(2.2)
    n += 1
    txt = "Hello %d there"%n
    lru.sendPacket(0xff, 0x21, txt.encode()) 
    pkt = lru.readPacket()
    o.fill(0)
    o.text('sent:%d' % (n), 0, 0)
    o.show()



Sent 13 lines (226 bytes) to main.py.


In [4]:
from LightLora import lorautil, spicontrol
lru = lorautil.LoraUtil()


Exception 868100000
